# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [1]:
import pandas as pd
import warnings
customer = pd.read_csv("customer_join.csv")
uselog_months= pd.read_csv("use_log_months.csv")

warnings.simplefilter('ignore')

In [2]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [4]:
year_months = list(uselog_months["年月"].unique())
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [5]:
tmp = uselog_months.loc[uselog_months["年月"] == year_months[1]]
tmp

,年月,customer_id,count
2991,201805,AS002855,5
2992,201805,AS009373,4
2993,201805,AS015233,7
2994,201805,AS015315,3
2995,201805,AS015739,5
...,...,...,...
6049,201805,TS993439,10
6050,201805,TS995299,7
6051,201805,TS999079,5
6052,201805,TS999231,4


In [6]:
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1,len(year_months)):
    # 直近1年のデータのうち、該当する年月のデータのみを引っ張ってくる
    tmp = uselog_months.loc[uselog_months["年月"] == year_months[i]]
    # 列名変更
    tmp.rename(columns ={"count":"count_0"},inplace = True)
    # 該当する年月の前月のデータを引っ張ってくる
    tmp_before = uselog_months.loc[uselog_months["年月"] == year_months[i-1]]
    # 年月の列を削除する
    del tmp_before["年月"]
    # 列名変更
    tmp_before.rename(columns={"count":"count_1"},inplace=True)
    # 結合
    tmp = pd.merge(tmp,tmp_before , on="customer_id",how="left")
    # 元のDFにくっつける
    uselog = pd.concat([uselog , tmp] ,ignore_index = True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [7]:
from dateutil.relativedelta import relativedelta
#　退会したユーザーのみのDFを用意する
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    # 退会日の１ヶ月前を抜き出す
    # iloc[i]をfor文で回すことで1行ずつ処理を実行する
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


退会したユーザーの顧客前月のデータのみなので欠損値が多数存在する<br>
データが欠損していないものだけを残してあとは除外する

In [8]:
exit_uselog = exit_uselog.dropna(subset=["name"]) 
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


退会者のみのデータ

### ノック43：継続顧客のデータを作成しようunique

In [9]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [10]:
conti_uselog = conti_uselog.sample(frac = 1).reset_index(drop = True) 
conti_uselog = conti_uselog.drop_duplicates(subset = "customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201808,TS999231,3,5.0,XXXX,C01,M,2017-03-01,NaN,CA1,...,オールタイム,10500.0,通常,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0
1,201812,AS036678,5,7.0,XXXXX,C01,M,2016-05-01,NaN,CA1,...,オールタイム,10500.0,通常,5.083333,5.5,8.0,2.0,1.0,2019-04-30,35.0
2,201808,PL527945,9,9.0,XXXXX,C01,F,2016-11-01,NaN,CA1,...,オールタイム,10500.0,通常,5.166667,4.0,9.0,3.0,1.0,2019-04-30,29.0
3,201808,IK516841,4,5.0,XXXXX,C01,F,2015-08-01,NaN,CA1,...,オールタイム,10500.0,通常,4.583333,5.0,6.0,3.0,1.0,2019-04-30,44.0
4,201809,HD314081,6,9.0,XXXX,C03,F,2017-10-01,NaN,CA1,...,ナイト,6000.0,通常,7.333333,7.5,10.0,5.0,1.0,2019-04-30,18.0


In [11]:
predict_data = pd.concat([conti_uselog,exit_uselog],ignore_index = True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201808,TS999231,3,5.0,XXXX,C01,M,2017-03-01,NaN,CA1,...,10500.0,通常,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0,NaT
1,201812,AS036678,5,7.0,XXXXX,C01,M,2016-05-01,NaN,CA1,...,10500.0,通常,5.083333,5.5,8.0,2.0,1.0,2019-04-30,35.0,NaT
2,201808,PL527945,9,9.0,XXXXX,C01,F,2016-11-01,NaN,CA1,...,10500.0,通常,5.166667,4.0,9.0,3.0,1.0,2019-04-30,29.0,NaT
3,201808,IK516841,4,5.0,XXXXX,C01,F,2015-08-01,NaN,CA1,...,10500.0,通常,4.583333,5.0,6.0,3.0,1.0,2019-04-30,44.0,NaT
4,201809,HD314081,6,9.0,XXXX,C03,F,2017-10-01,NaN,CA1,...,6000.0,通常,7.333333,7.5,10.0,5.0,1.0,2019-04-30,18.0,NaT


### ノック44：予測する月の在籍期間を作成しよう

In [12]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"] , format = "%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i],predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years * 12 + delta.months)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201808,TS999231,3,5.0,XXXX,C01,M,2017-03-01,NaN,CA1,...,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0,NaT,17,2018-08-01
1,201812,AS036678,5,7.0,XXXXX,C01,M,2016-05-01,NaN,CA1,...,5.083333,5.5,8.0,2.0,1.0,2019-04-30,35.0,NaT,31,2018-12-01
2,201808,PL527945,9,9.0,XXXXX,C01,F,2016-11-01,NaN,CA1,...,5.166667,4.0,9.0,3.0,1.0,2019-04-30,29.0,NaT,21,2018-08-01
3,201808,IK516841,4,5.0,XXXXX,C01,F,2015-08-01,NaN,CA1,...,4.583333,5.0,6.0,3.0,1.0,2019-04-30,44.0,NaT,36,2018-08-01
4,201809,HD314081,6,9.0,XXXX,C03,F,2017-10-01,NaN,CA1,...,7.333333,7.5,10.0,5.0,1.0,2019-04-30,18.0,NaT,11,2018-09-01


### ノック45：欠損値を除去しよう

In [13]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               250
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [29]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2644
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2644
period                  0
now_date                0
dtype: int64

In [15]:
predicted_data.drop(["preriod"],axis = 1,inplace = True)

KeyError: "['preriod'] not found in axis"

### ノック46：文字列型の変数を処理できるように整形しよう

In [30]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3696 entries, 0 to 3945
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   年月                 3696 non-null   object        
 1   customer_id        3696 non-null   object        
 2   count_0            3696 non-null   int64         
 3   count_1            3696 non-null   float64       
 4   name               3696 non-null   object        
 5   class              3696 non-null   object        
 6   gender             3696 non-null   object        
 7   start_date         3696 non-null   datetime64[ns]
 8   end_date           1052 non-null   object        
 9   campaign_id        3696 non-null   object        
 10  is_deleted         3696 non-null   float64       
 11  class_name         3696 non-null   object        
 12  price              3696 non-null   float64       
 13  campaign_name      3696 non-null   object        
 14  mean    

In [31]:
target_col = ["campaign_name","class_name","gender","count_1","routine_flg","period","is_deleted"]
predict_data_limit = predict_data[target_col]
predict_data_limit.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,M,5.0,1.0,17,0.0
1,通常,オールタイム,M,7.0,1.0,31,0.0
2,通常,オールタイム,F,9.0,1.0,21,0.0
3,通常,オールタイム,F,5.0,1.0,36,0.0
4,通常,ナイト,F,9.0,1.0,11,0.0


In [32]:
predict_data_dummy = pd.get_dummies(predict_data_limit)
predict_data_dummy.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,17,0.0,0,0,1,1,0,0,0,1
1,7.0,1.0,31,0.0,0,0,1,1,0,0,0,1
2,9.0,1.0,21,0.0,0,0,1,1,0,0,1,0
3,5.0,1.0,36,0.0,0,0,1,1,0,0,1,0
4,9.0,1.0,11,0.0,0,0,1,0,0,1,1,0


In [33]:
del predict_data_dummy["campaign_name_通常"]
del predict_data_dummy["class_name_ナイト"]
del predict_data_dummy["gender_M"]
predict_data_dummy.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,17,0.0,0,0,1,0,0
1,7.0,1.0,31,0.0,0,0,1,0,0
2,9.0,1.0,21,0.0,0,0,1,0,1
3,5.0,1.0,36,0.0,0,0,1,0,1
4,9.0,1.0,11,0.0,0,0,0,0,1


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [34]:
predict_data_dummy.isna().sum()

count_1                0
routine_flg            0
period                 0
is_deleted             0
campaign_name_入会費半額    0
campaign_name_入会費無料    0
class_name_オールタイム      0
class_name_デイタイム       0
gender_F               0
dtype: int64

In [27]:
predict_data_dummy["count_1"].value_counts()

4.0     593
5.0     590
6.0     525
3.0     457
7.0     425
8.0     306
2.0     296
9.0     196
1.0     129
10.0    107
11.0     44
12.0     20
13.0      7
14.0      1
Name: count_1, dtype: int64

In [26]:
predict_data_dummy["count_1"] = predict_data_dummy["count_1"].astype(int)
predict_data_dummy["routine_flg"] = predict_data_dummy["routine_flg"].astype(int)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [35]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data_dummy.loc[predict_data_dummy["is_deleted"]==1]
conti = predict_data_dummy.loc[predict_data_dummy["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit,conti],ignore_index = True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state = 0)
model.fit(X_train,y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1.
 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1.

In [36]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
169,1.0,1.0
902,1.0,1.0
1815,0.0,0.0
117,1.0,1.0
1020,1.0,1.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [37]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.8878326996197718


In [38]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8878326996197718
0.976552598225602


In [39]:
X = pd.concat([exit,conti], ignore_index = True)
y = X["is_deleted"]
del X["is_deleted"]
X_train,X_test,y_train,y_test =sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0,max_depth=5)
model.fit(X_train,y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9201520912547528
0.9252217997465145


### ノック49：モデルに寄与している変数を確認しよう

In [41]:
importance = pd.DataFrame({"feature_names":X.columns,"coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.341605
1,routine_flg,0.128722
2,period,0.529673
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.000000
7,gender_F,0.000000


### ノック50：顧客の退会を予測しよう

In [42]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [43]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [44]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]
